# Data Augmentation with ResNet18 

In [1]:
#import necessary libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
import matplotlib.pyplot as plt
import numpy as np

#ici ça normalise 

In [2]:
#normalize
import torch
from torchvision import datasets, transforms as T
from torch.utils.data import DataLoader
from tqdm import tqdm  # pour la barre de progression

base_dir = "ipeo_hurricane_for_students"

# juste transformer en tensor pour calcul
transform = T.Compose([T.ToTensor()])

train_ds = datasets.ImageFolder(f"{base_dir}/train", transform=transform)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=False, num_workers=4)

mean = 0.
std = 0.
nb_samples = 0

for imgs, _ in tqdm(train_loader, desc="Calcul mean/std"):
    batch_samples = imgs.size(0)
    imgs = imgs.view(batch_samples, imgs.size(1), -1)  # aplatir H*W pour chaque canal
    mean += imgs.mean(2).sum(0)
    std += imgs.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std  /= nb_samples

print("mean:", mean)
print("std:", std)


Calcul mean/std: 100%|██████████| 297/297 [02:10<00:00,  2.28it/s]

mean: tensor([0.3518, 0.3832, 0.2849])
std: tensor([0.1137, 0.0985, 0.0997])


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
#recuperation mean et std
normalize = T.Normalize(mean,
                        std)

transforms_train = T.Compose([
    T.RandomResizedCrop(224, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=15),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    T.RandomGrayscale(p=0.1),
    T.RandomApply([T.GaussianBlur(kernel_size=3)], p=0.3),
    T.ToTensor(),
    normalize
])
transforms_val = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    normalize
])
base_dir = "ipeo_hurricane_for_students"
train_ds = datasets.ImageFolder(os.path.join(base_dir, "train"), transform=transforms_train)
val_ds   = datasets.ImageFolder(os.path.join(base_dir, "validation"), transform=transforms_val)
test_ds  = datasets.ImageFolder(os.path.join(base_dir, "test"), transform=transforms_val)


model=resnet18